<a href="https://colab.research.google.com/github/LangqingZou/Math-509/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import pylab as pl
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier as DNN
from sklearn.tree import DecisionTreeRegressor

In [225]:
data = pd.read_csv('drive/MyDrive/Math_509_Project/data/data5.csv')
data.head()

,ST004D01T,IC001Q04TA,ESCS,WEALTH,EMOSUPS,GFOFAIL,SWBP,RESILIENCE,BELONG,weight
0,1,2.0,-1.9067,-1.3354,-0.6576,1.1422,-0.5928,-0.8547,-1.3850,8.44013
1,1,2.0,-1.9768,-1.3359,-1.3914,0.4637,0.7301,-0.8738,-0.6765,8.44013
2,1,1.0,-2.5606,-1.4253,-0.6576,1.3423,-1.3186,-1.0974,-1.0425,8.44013
3,2,1.0,-0.3868,-1.1210,-1.5824,1.1422,-1.8489,-1.2865,-1.0654,7.47709
4,2,1.0,-1.2306,-0.9500,-0.6576,-0.6870,-0.5928,-0.0614,-0.4450,7.47709


In [226]:
df = pd.DataFrame(data)
df = df.loc[df['IC001Q04TA'].isin([1,2,3])]

In [153]:
data.isnull().sum()

ST004D01T     0
IC001Q04TA    0
ESCS          0
WEALTH        0
EMOSUPS       0
GFOFAIL       0
SWBP          0
RESILIENCE    0
BELONG        0
weight        0
dtype: int64

In [215]:
df['ST004D01T'] = df['ST004D01T'].astype(object)
df['IC001Q04TA'] = df['IC001Q04TA'].astype(object)

In [216]:
dfDummies = pd.get_dummies(df[['ST004D01T','IC001Q04TA']],drop_first=True)
df = pd.concat([df, dfDummies], axis=1)
df.drop(['ST004D01T', 'IC001Q04TA'], axis=1)

,ESCS,WEALTH,EMOSUPS,GFOFAIL,SWBP,RESILIENCE,BELONG,weight,ST004D01T_2,IC001Q04TA_2.0,IC001Q04TA_3.0
0,-1.9067,-1.3354,-0.6576,1.1422,-0.59280,-0.85470,-1.3850,8.44013,0,1,0
1,-1.9768,-1.3359,-1.3914,0.4637,0.73010,-0.87380,-0.6765,8.44013,0,1,0
2,-2.5606,-1.4253,-0.6576,1.3423,-1.31860,-1.09740,-1.0425,8.44013,0,0,0
3,-0.3868,-1.1210,-1.5824,1.1422,-1.84890,-1.28650,-1.0654,7.47709,1,0,0
4,-1.2306,-0.9500,-0.6576,-0.6870,-0.59280,-0.06140,-0.4450,7.47709,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
61421,-2.0073,-2.2580,-1.2973,-0.1300,0.39734,0.34906,-0.3258,282.26599,1,0,0
61422,-0.9587,-1.6809,0.2127,-0.9295,0.39734,0.34906,-0.2812,282.26599,0,0,0
61423,-3.1772,-3.8509,-0.6576,-0.1300,0.39734,0.34906,0.1403,282.26599,0,0,0
61424,-1.7318,-2.8136,-1.4902,0.1097,0.39734,0.34906,-0.6115,282.26599,1,0,0


In [167]:
# df.to_csv("data7.csv")

In [217]:
columns = ['ST004D01T_2','IC001Q04TA_2.0','IC001Q04TA_3.0','ESCS','WEALTH','EMOSUPS','SWBP','RESILIENCE','BELONG']
X = df[['ST004D01T_2','IC001Q04TA_2.0','IC001Q04TA_3.0','ESCS','WEALTH','EMOSUPS','SWBP','RESILIENCE','BELONG','weight']]
Y = df['GFOFAIL']

In [218]:
# standarlized the dataset
scaler = StandardScaler()
X[columns] = scaler.fit_transform(df[columns])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [197]:
X

,ST004D01T_2,IC001Q04TA_2.0,IC001Q04TA_3.0,ESCS,WEALTH,EMOSUPS,SWBP,RESILIENCE,BELONG,weight
0,-0.996452,7.041480,-0.176032,-1.239785,-0.547672,-0.584552,-0.735280,-0.843958,-1.425531,8.44013
1,-0.996452,7.041480,-0.176032,-1.301676,-0.548154,-1.372284,0.720455,-0.865388,-0.588830,8.44013
2,-0.996452,-0.142016,-0.176032,-1.817110,-0.634277,-0.584552,-1.533959,-1.116264,-1.021056,8.44013
3,1.003561,-0.142016,-0.176032,0.102128,-0.341131,-1.577322,-2.117508,-1.328431,-1.048100,7.47709
4,1.003561,-0.142016,-0.176032,-0.642859,-0.176399,-0.584552,-0.735280,0.046112,-0.315440,7.47709
...,...,...,...,...,...,...,...,...,...,...
61421,1.003561,-0.142016,-0.176032,-1.328604,-1.436455,-1.271268,0.354282,0.506641,-0.174672,282.26599
61422,-0.996452,-0.142016,-0.176032,-0.402800,-0.880508,0.349712,0.354282,0.506641,-0.122001,282.26599
61423,-0.996452,-0.142016,-0.176032,-2.361504,-2.970969,-0.584552,0.354282,0.506641,0.375768,282.26599
61424,1.003561,-0.142016,-0.176032,-1.085366,-1.971690,-1.478345,0.354282,0.506641,-0.512068,282.26599


In [219]:
# split into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(X, Y.astype('float'), test_size=0.20, random_state=53)
# y_train = y_train.str.strip()
# y_test = y_test.astype(str).strip().to_numpy()


In [220]:
x_train

,ST004D01T_2,IC001Q04TA_2.0,IC001Q04TA_3.0,ESCS,WEALTH,EMOSUPS,SWBP,RESILIENCE,BELONG,weight
4530,1.003561,-0.142016,-0.176032,-0.665108,-0.663755,-0.584552,0.720455,0.046112,-0.165933,7.01081
54981,-0.996452,-0.142016,-0.176032,-0.114269,-0.643429,-0.584552,1.280015,0.046112,-0.956577,94.26382
10639,1.003561,-0.142016,-0.176032,0.361876,0.929233,-0.584552,-0.735280,-0.759361,1.015251,241.25369
44860,-0.996452,-0.142016,-0.176032,1.135293,1.261202,-2.505252,-0.820562,2.773319,-0.953742,6.95625
39140,1.003561,-0.142016,-0.176032,-0.141286,-0.287280,-1.022646,0.122711,-0.825895,-1.017395,23.33189
...,...,...,...,...,...,...,...,...,...,...
47099,-0.996452,-0.142016,-0.176032,1.299864,1.986891,-0.857864,0.705688,0.547863,0.616445,4.44705
56090,-0.996452,-0.142016,-0.176032,-1.376280,-1.114987,-0.584552,0.354282,0.506641,-0.122001,147.35958
35936,-0.996452,-0.142016,-0.176032,0.304311,-0.025539,-0.584552,-0.735280,-1.549126,-0.956577,24.38717
58514,1.003561,-0.142016,-0.176032,-1.446382,-1.875163,0.349712,0.354282,0.506641,-0.221791,190.67970


# Linear Regression Model

In [208]:
## construct linear regression model
clf = LinearRegression().fit(x_train[columns],y_train,x_train['weight'])
 
## predicted values
y_pred = clf.predict(x_test[columns])

In [209]:
print("coefficients =",clf.coef_)    # coefficients
print("intercept b =",clf.intercept_) # intercept
print('Mean Absolute Error =', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error =', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error =', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 Score:', r2_score(y_test, y_pred))

coefficients = [-0.10360924 -0.00576155  0.00619346  0.07105544  0.02021214  0.04201135
 -0.0479567  -0.08758022 -0.13756272]
intercept b = 0.18768504782125092
Mean Absolute Error = 0.696632759181914
Mean Squared Error = 0.7972099525499642
Root Mean Squared Error = 0.8928661448111717
R2 Score: 0.056035089694298046


# Lasso Regression Model

In [210]:
lasso=Lasso(alpha=0.01,max_iter=10000)
lasso.fit(x_train,y_train)
print("Lasso training dataset score："+str(r2_score(y_train,lasso.predict(x_train))))
print("Lasso testing dataset score："+str(r2_score(y_test,lasso.predict(x_test))))

Lasso training dataset score：0.07000160903257246
Lasso testing dataset score：0.062137062735843074


# Ridge Regression Model

In [212]:
ridge = Ridge(alpha=0.5)
ridge.fit(x_train, y_train)
print("Ridge training dataset score："+str(r2_score(y_train,ridge.predict(x_train))))
print("Ridge testing dataset score："+str(r2_score(y_test,ridge.predict(x_test))))

Ridge training dataset score：0.07093168195425181
Ridge testing dataset score：0.06281988650067194


# Desicion Tree

# Neural Network

In [222]:
from sklearn import preprocessing
# split into training and testing datasets
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=53)
label_encoder = preprocessing.LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [223]:
y_train

array([ 32, 411, 118, ..., 411, 411,  65])

In [224]:
# hidden_layer_sizes=(2, 1) hidden层2层,第一层2个神经元，第二层1个神经元)，2层隐藏层，也就有3层神经网络
nn = DNN(solver='sgd', alpha=0.01,hidden_layer_sizes=(2, 1), random_state=1)
nn.fit(x_train, y_train)
print("Neural networking training dataset score："+str(r2_score(y_train,nn.predict(x_train))))
print("Neural networking testing dataset score："+str(r2_score(y_test,nn.predict(x_test))))

Neural networking training dataset score：-1.968870028519258
Neural networking testing dataset score：0.06281988650067194


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
regressor = DecisionTreeRegressor(random_state=0)
cross_val_score(regressor, x_train, y_train, cv=10)

array([-0.29801955, -0.32859603, -0.30088056, -0.31872497, -0.32041823,
       -0.3242865 , -0.33204199, -0.31689387, -0.32321852, -0.31911775])

In [ ]:
# Import necessary modules
from scipy.stats import randint
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV

# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [3, 6, 9],
              "min_samples_leaf": randint(1, 10)}

# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeRegressor()

# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

# Fit it to the data
tree_cv.fit(x_train, y_train)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

decisiontree_pred = tree_cv.predict(x_test)
r2 = tree_cv.score(x_test, y_test)
rmse = mean_squared_error(y_test, decisiontree_pred, squared = False)
print("Tuned Decision Tree R squared: {}".format(r2))
print("Tuned Decision Tree RMSE: {}".format(rmse))

Tuned Decision Tree Parameters: {'max_depth': 3, 'min_samples_leaf': 4}
Best score is 0.0005080794529318755
Tuned Decision Tree R squared: 0.0005361267796762936
Tuned Decision Tree RMSE: 0.9910021296184451
